<a href="https://colab.research.google.com/github/honginhwa/pytorch_study/blob/master/resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.optim import lr_scheduler

import torchvision
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms

import random as rand
from random import *
import os
import cv2
import copy
import time
from PIL import Image

# 첫 번째 시도(No Normalize & No Augmentation)
### 데이터셋 정의 및 transforms

In [3]:
train_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [4]:
data_dir = "/content/drive/My Drive/Colab Notebooks/Labeld_Crop_Data"
trDsets = {x: dset.ImageFolder(os.path.join(data_dir, x), train_transforms[x]) for x in ['train', 'val']}
trLoaders = {x: torch.utils.data.DataLoader(trDsets[x], batch_size=64, shuffle=True, num_workers=4) for x in ['train', 'val']}

teDsets = dset.ImageFolder(os.path.join(data_dir,'test'), transform=test_transforms)
teLoaders = torch.utils.data.DataLoader(teDsets, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
print(trDsets['train'].classes)
print(teDsets.classes)

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']
['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


여기서 틀린 부분을 알아챔..ㅠ  
train class가 0, 1, 10 으로 시작하기 때문에, 이전에 만들어놨던 answer csv의 경우도 이 순서대로 했어야 했음..

In [ ]:
trDsets_sizes = {x: len(trDsets[x]) for x in ['train', 'val']}
print(trDsets_sizes)
class_names = trDsets['train'].classes
print(class_names)

{'train': 9936, 'val': 2503}
['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']


### GPU 사용

In [ ]:
print(torch.cuda.is_available())     # GPU 사용 가능 여부
print(torch.cuda.current_device())   # GPU 디바이스의 위치
print(torch.cuda.device_count())     # 사용가능한 GPU 개수
print(torch.cuda.get_device_name(0)) # GPU의 이름
print(torch.cuda.device(0))

True
0
1
Tesla T4


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

### 모델 생성 - ResNet18

In [ ]:
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
print(num_ftrs)
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)
print(model)

512
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=0.1, patience=11)

### 모델 훈련

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=8):
    
    global_info = []
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict()) # state_dict() = 모델 불러오기 
    best_acc = 0.0
    early_stopping = EarlyStopping(patience=11, verbose=True)  # EarlyStopping = 11번에 큰 변화가 없다면 학습을 그만한다.
    for epoch in range(num_epochs):
        local_info = []
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)                                            #train 시작

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()                                   # list에 train이면 학습 val이면 평가 
            else:
                model.eval()                              
                
#                if epoch > 0:
#                    scheduler.step(val_loss)
                    
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in trLoaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()                              # 가중치 변화도 = 0 >> forward 

                # forward
                # tensor는 인풋값이 디폴트는 그래디언트가 안들어가있는데 이 값을  
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()                              # backward로 최신화로 스텝으로 가자 

                running_loss += loss.item() * inputs.size(0)          # item() = tensor값을 scalar값으로 표기      
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / trDsets_sizes[phase]
            if phase == 'val':
                val_loss = running_loss / trDsets_sizes['val']
            epoch_acc = running_corrects.double() / trDsets_sizes[phase]

            if phase == 'train':
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)
            else:
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        lr_get = get_lr(optimizer)
        print("Current learning rate : {:.8f}".format(lr_get))
        global_info.append(local_info)
        
        if phase =='val':
            early_stopping(epoch_loss, model)

            if early_stopping.early_stop:
                print("Early stopping")
                break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
torch.cuda.empty_cache()
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

Epoch 0/99
----------
train Loss: 0.5153 Acc: 0.8737
val Loss: 0.9009 Acc: 0.6968
Current learning rate : 0.00100000
Validation loss decreased (inf --> 0.900943).  Saving model ...
Epoch 1/99
----------
train Loss: 0.3251 Acc: 0.9168
val Loss: 1.0228 Acc: 0.7004
Current learning rate : 0.00100000
EarlyStopping counter: 1 out of 11
Epoch 2/99
----------
train Loss: 0.2463 Acc: 0.9358
val Loss: 0.5974 Acc: 0.8386
Current learning rate : 0.00100000
Validation loss decreased (0.900943 --> 0.597357).  Saving model ...
Epoch 3/99
----------
train Loss: 0.2020 Acc: 0.9462
val Loss: 0.3426 Acc: 0.9085
Current learning rate : 0.00100000
Validation loss decreased (0.597357 --> 0.342562).  Saving model ...
Epoch 4/99
----------
train Loss: 0.1574 Acc: 0.9562
val Loss: 2.4803 Acc: 0.5693
Current learning rate : 0.00100000
EarlyStopping counter: 1 out of 11
Epoch 5/99
----------
train Loss: 0.1308 Acc: 0.9662
val Loss: 0.4910 Acc: 0.8466
Current learning rate : 0.00100000
EarlyStopping counter: 2 o

In [ ]:
torch.save(model_ft, 'lotte_model_resnet18_v2.pt')

### 모델 테스트

In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

teDiter = iter(teLoaders) # 배치 1개씩 부른다.
images, labels = teDiter.next()
imshow(torchvision.utils.make_grid(images)) # 여러 이미지를 모아 하나의 이미지로 만들 수 잇다. 
print('GroundTruth: ', ' '.join('%5s' % class_names[labels[j]] for j in range(4)))

In [ ]:
testModel = torch.load('lotte_model_resnet18_v2.pt', map_location=device)

#### 전체 테스트 결과

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = testModel(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test images: %d %%' % (100 * correct / total))

#### 클래스별 테스트 결과

In [ ]:
tag_classes = ['ID_gum', 'buttering', 'couque_coffee', 'chocopie', 'cidar', 'couque_white', 'coke', 'diget_ori', 'diget_choco', 'gumi_gumi', 'homerunball', 'jjolbyung_noodle', 'juicyfresh', 'jjolbyung_ori', 'spearmint', 'squid_peanut', 'samdasu', 'tuna', 'toreta', 'vita500', 'welchs', 'zec']
tag_dict = dict()
for i, label in enumerate(tag_classes):
    tag_dict[i] = label

print(tag_dict)

In [ ]:
class_correct = list(0 for i in range(len(class_names)))
class_total = list(0 for i in range(len(class_names)))
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = model_ft(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()

        for i in range(c.size(0)):
            label = labels[i]
            class_correct[int(label.item())] += c[i].item()
            class_total[int(label.item())] += 1

for i in range(len(class_names)):
    print('Accuracy of %5s : %2d %%' % (tag_dict[int(class_names[i])], 100 * class_correct[i] / class_total[i]))

# 두 번째 시도(Normalize & No Augmentation)
### 데이터셋 정의 및 transforms

normalized 된 것과 안된 것을 비교해보자.

In [ ]:
testPath = "/content/drive/My Drive/Colab Notebooks/Resize_Crop_Data_2"
testDir = dset.ImageFolder(os.path.join(testPath, 'train'))
numList = [i for i in range(len(testDir))]

In [ ]:
def display_augmented_images(customTransform):
    fig, subplots = plt.subplots(2, 5, figsize=(13, 6))
    
    for i in range(5):
        axi1 = subplots.flat[i]
        axi2 = subplots.flat[i+5]

        ori_img = testDir[randSample[i]][0]
        trf_img = customTransform(ori_img)

        axi1.imshow(ori_img)
        axi2.imshow(transforms.functional.to_pil_image(trf_img))
        axi1.set_title('original')
        axi2.set_title('transformed')

In [ ]:
randSample = rand.sample(numList, 5)
norm1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
display_augmented_images(norm1)

In [ ]:
train_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
data_dir = "/content/drive/My Drive/Colab Notebooks/Labeld_Crop_Data"
trDsets = {x: dset.ImageFolder(os.path.join(data_dir, x), train_transforms[x]) for x in ['train', 'val']}
trLoaders = {x: torch.utils.data.DataLoader(trDsets[x], batch_size=64, shuffle=True, num_workers=4) for x in ['train', 'val']}

teDsets = dset.ImageFolder(os.path.join(data_dir, 'test'), transform=test_transforms)
teLoaders = torch.utils.data.DataLoader(teDsets, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
trDsets_sizes = {x: len(trDsets[x]) for x in ['train', 'val']}
class_names = trDsets['train'].classes

### 모델 생성

In [ ]:
device2 = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [ ]:
model2 = models.resnet18(pretrained=False)
num_ftrs = model2.fc.in_features
model2.fc = nn.Linear(num_ftrs, len(class_names))
model2 = model2.to(device2)
print(model2)

위에서 학습할 때, patience가 10인 상태에서 더 좋은 결과를 냈기 때문에 조금 늘려주자.

In [ ]:
criterion2 = nn.CrossEntropyLoss()
optimizer_ft2 = optim.SGD(model2.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler2 = lr_scheduler.ReduceLROnPlateau(optimizer_ft2, factor=0.1, patience=15)

### 모델 훈련

In [ ]:
torch.cuda.empty_cache()
model_ft2 = train_model(model2, criterion2, optimizer_ft2, exp_lr_scheduler2, num_epochs=100)

In [ ]:
torch.save(model_ft2, 'lotte_model_resnet18_v3.pt')

### 모델 테스트

In [ ]:
testModel2 = torch.load('lotte_model_resnet18_v3.pt', map_location=device2)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = testModel2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test images: %d %%' % (100 * correct / total))

#### 클래스별 테스트

In [ ]:
class_correct = list(0 for i in range(len(class_names)))
class_total = list(0 for i in range(len(class_names)))
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = model_ft2(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()

        for i in range(c.size(0)):
            label = labels[i]
            class_correct[int(label.item())] += c[i].item()
            class_total[int(label.item())] += 1

for i in range(len(class_names)):
    print('Accuracy of %5s : %2d %%' % (tag_dict[int(class_names[i])], 100 * class_correct[i] / class_total[i]))

# 세 번째 시도(No Normalize & Augmentation)
### 데이터셋 정의 및 transforms

필요한 augmentation 기법만 골라서 사용해보자.

In [ ]:
def display_augmented_images(customTransform):
    fig, subplots = plt.subplots(2, 5, figsize=(13, 6))
    
    for i in range(5):
        axi1 = subplots.flat[i]
        axi2 = subplots.flat[i+5]

        ori_img = testDir[randSample[i]][0]
        trf_img = customTransform(ori_img)

        axi1.imshow(ori_img)
        axi2.imshow(trf_img)
        axi1.set_title('original')
        axi2.set_title('transformed')

#### Random Horizontal Flip

In [ ]:
randSample = rand.sample(numList, 5)
horiz_flip = transforms.RandomHorizontalFlip(p=1)
display_augmented_images(horiz_flip)

#### Random Vertical Flip

In [ ]:
randSample = rand.sample(numList, 5)
vert_flip = transforms.RandomVerticalFlip(p=1)
display_augmented_images(vert_flip)

#### Random Affine

In [ ]:
randSample = rand.sample(numList, 5)
affine = transforms.RandomAffine(30)
display_augmented_images(affine)

#### Random Crop

In [ ]:
randSample = rand.sample(numList, 5)
crop = transforms.RandomCrop((200, 200))
display_augmented_images(crop)

#### Random Perspective

In [ ]:
randSample = rand.sample(numList, 5)
perspect = transforms.RandomPerspective(p=1)
display_augmented_images(perspect)

#### Random Rotation

In [ ]:
randSample = rand.sample(numList, 5)
rot = transforms.RandomRotation(90)
display_augmented_images(rot)

#### Color Jitter(Brightness)

In [ ]:
randSample = rand.sample(numList, 5)
color = transforms.ColorJitter(brightness=(0.5, 1.5))
display_augmented_images(color)

#### Color Jitter(Contrast)

In [ ]:
randSample = rand.sample(numList, 5)
color = transforms.ColorJitter(contrast=(0.5, 3))
display_augmented_images(color)

#### Color Jitter(Saturation)

In [ ]:
randSample = rand.sample(numList, 5)
color = transforms.ColorJitter(saturation=(0.5, 3))
display_augmented_images(color)

#### Color Jitter(Hue)

In [ ]:
randSample = rand.sample(numList, 5)
color = transforms.ColorJitter(hue=(-0.5, 0.5))
display_augmented_images(color)

#### Pad

In [ ]:
randSample = rand.sample(numList, 5)
pad = transforms.Pad(padding=10)
display_augmented_images(pad)

꽤 쓸만한 augmentation 기법  
- Random Horizontal Flip
- Random Vertical Flip
- Color Jitter(Brightness)
- Color Jitter(Contrast)
- Color Jitter(Saturation)

In [ ]:
# 위의 기법들을 확률적으로 선택해서 적용하는 transforms를 만들어보자.
train_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomChoice([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ColorJitter(brightness=(0.5, 1.5)),
            transforms.ColorJitter(contrast=(0.5, 3)),
            transforms.ColorJitter(saturation=(0.5, 3)),
        ]),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [ ]:
data_dir = "../Data_Set/Labeld_Crop_Data/"
trDsets = {x: dset.ImageFolder(os.path.join(data_dir, x), train_transforms[x]) for x in ['train', 'val']}
trLoaders = {x: torch.utils.data.DataLoader(trDsets[x], batch_size=64, shuffle=True, num_workers=4) for x in ['train', 'val']}

teDsets = dset.ImageFolder(os.path.join(data_dir, 'test'), transform=test_transforms)
teLoaders = torch.utils.data.DataLoader(teDsets, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
trDsets_sizes = {x: len(trDsets[x]) for x in ['train', 'val']}
class_names = trDsets['train'].classes

### 모델 생성

In [ ]:
device3 = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [ ]:
model3 = models.resnet18(pretrained=False)
num_ftrs = model3.fc.in_features
model3.fc = nn.Linear(num_ftrs, len(class_names))
model3 = model3.to(device3)
print(model3)

In [ ]:
criterion3 = nn.CrossEntropyLoss()
optimizer_ft3 = optim.SGD(model3.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler3 = lr_scheduler.ReduceLROnPlateau(optimizer_ft3, factor=0.1, patience=15)

### 모델 훈련

In [ ]:
torch.cuda.empty_cache()
model_ft3 = train_model(model3, criterion3, optimizer_ft3, exp_lr_scheduler3, num_epochs=100)

### 모델 테스트

In [ ]:
torch.save(model_ft3, 'lotte_model_resnet18_v4.pt')

In [ ]:
testModel3 = torch.load('lotte_model_resnet18_v4.pt', map_location=device3)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = testModel3(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test images: %d %%' % (100 * correct / total))

#### 클래스별 테스트

In [ ]:
class_correct = list(0 for i in range(len(class_names)))
class_total = list(0 for i in range(len(class_names)))
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = testModel3(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()

        for i in range(c.size(0)):
            label = labels[i]
            class_correct[int(label.item())] += c[i].item()
            class_total[int(label.item())] += 1

for i in range(len(class_names)):
    print('Accuracy of %5s : %2d %%' % (tag_dict[int(class_names[i])], 100 * class_correct[i] / class_total[i]))